# Probability Distributions, Gaussian and Uniform
stough 202-

In the [matplotlib tutorial](./matplotlib_tutorial.ipynb) we saw how to produce a histogram of color distribution in an image. These histograms represent the [probability distribution](https://en.wikipedia.org/wiki/Probability_distribution) on color; that is, the probability of a particular color in an image corresponds to the height of the histogram at that color (given the histogram of that image). 

In this demo we'll look at Uniform and Gaussian probability distributions, and what corresponding "random" images look like.

- [Uniform](#uniform)
- [Gaussian](#gaussian)

## Imports
We'll produce random numbers using numpy's updated random-number-generation techniques. Read more about it [here](https://numpy.org/doc/stable/reference/random/index.html).

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils

In [ ]:
from numpy.random import default_rng
rng = default_rng() # could add parameter "seed="

In [ ]:
rng

At this point the object `rng` is our random number generator, and it has been seeded (with the current time for example) to produce different random sequences each run through this notebook. You could also seed the default random number generator to ensure the same random sequence (good for debugging). 

<a id='uniform'></a>
## Uniform Random Distribution 
When you think of the word "random," you probably mean that any outcome of the "random" process is equally likely. Let's say we're generating integers between 0 and 255 (inclusive). We would expect over many samples that no number is a lot more likely than any other. 

In [ ]:
X = rng.integers(0, high=256, size=2**20)

In [ ]:
plt.figure(figsize=(6,3))
n, bins, _ = plt.hist(X, bins = np.arange(257));

In [ ]:
matrix_utils.arr_info(n)

In [ ]:
n.mean()

In the above we generate about 1M samples in the range [0,255]. In fact we generate $2**20 = 1048576$. Since these random numbers are split among $256 = 2**8$ possibilites, that means on average we should get $2**12 = 4096$ hits for each output integer. What makes the randomness *uniform* is that most of the bins have about the same number of outcomes, hovering near 4K as expected.  

- Little complication in generating the plot: [`plt.hist`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.hist.html) expects the `bins` argument to include both left end of the first bin *and* the right end of the last bin. Since [`arange`](https://numpy.org/doc/stable/reference/generated/numpy.arange.html) provides the sequence $[0, N)$ exclusive, we call it with 257 to make sure we get all the way to $[0,...,256]$.

Now let's generate an image of uniform random colors

In [ ]:
I_uniform = np.stack([rng.integers(0,256,2**14).reshape(2**7,2**7) for i in range(3)], axis=2)

In [ ]:
plt.figure(figsize=(3,3))
plt.imshow(I_uniform);

The above is what a truly random image looks like. 

- Is it anyhing like the images capture, communicate to each other, attempt to study? 
- What is missing from it?
- What does this say about the kinds of images that we actually care about?

<a id='gaussian'></a>
## Gaussian Random Distribution
